<a href="https://colab.research.google.com/github/sungchan1/goingSaboho/blob/gwangseok/credit_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/content/drive/MyDrive/colab/data/creditcard.csv')
std_scaler = StandardScaler()

df.insert(29, 'scaled_amount', std_scaler.fit_transform(df['Amount'].values.reshape(-1,1)))
df.drop(['Amount'], axis=1, inplace=True)

In [ ]:
cd "/content/drive/My Drive/Credit_fraud"

In [ ]:
!git config --global user.email 'hks8946@gmail.com'
!git config --global user.name 'c0natus'

In [ ]:
!git push -u origin gwangseok